**라이브러리 로드**

In [ ]:
import torch
print("torch.cuda.is_available() =", torch.cuda.is_available())
print("torch.cuda.device_count() =", torch.cuda.device_count())
print("torch.version.cuda =", torch.version.cuda)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

***라이브러리 설치 - RoboFlow***

In [ ]:
!pip install roboflow ultralytics -q

**Roboflow 데이터 다운로드**

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="     ")
project = rf.workspace("yunhoo93").project("dataset-bm9aj")
dataset = project.version(9).download("yolov8")

**YOLO Fine-tuning**

In [ ]:
from ultralytics import YOLO
from roboflow import Roboflow
import matplotlib.pyplot as plt
import os, random, cv2

# Roboflow 데이터셋 로드
train_img_dir = os.path.join(dataset.location, "train", "images")
valid_img_dir = os.path.join(dataset.location, "valid", "images")
test_img_dir  = os.path.join(dataset.location, "test", "images")

train_lbl_dir = os.path.join(dataset.location, "train", "labels")
valid_lbl_dir = os.path.join(dataset.location, "valid", "labels")
test_lbl_dir  = os.path.join(dataset.location, "test", "labels")



# 클래스별 이미지 하나씩 선택 (랜덤 버전)
def get_one_image_per_class(image_dir, label_dir, max_count=4):
    class_to_imgs = {}

    for lbl in os.listdir(label_dir):
        if not lbl.endswith(".txt"):
            continue

        label_path = os.path.join(label_dir, lbl)
        with open(label_path, "r") as f:
            line = f.readline().strip()

        if not line:
            continue

        cls_id = int(line.split()[0])

        img_name = lbl.replace(".txt", ".jpg")
        img_path = os.path.join(image_dir, img_name)

        if os.path.exists(img_path):
            if cls_id not in class_to_imgs:
                class_to_imgs[cls_id] = []
            class_to_imgs[cls_id].append(img_path)

    selected = {}
    for cls_id in sorted(class_to_imgs.keys()):
        if len(selected) >= max_count:
            break
        selected[cls_id] = random.choice(class_to_imgs[cls_id])

    return selected


# 시각화 함수 (4개 클래스 다른 이미지)
def show_by_class(image_dir, label_dir, title):
    cls_imgs = get_one_image_per_class(image_dir, label_dir, max_count=4)
    class_ids = sorted(cls_imgs.keys())

    plt.figure(figsize=(12, 10))
    plt.suptitle(title, fontsize=18)

    for idx, cls_id in enumerate(class_ids, 1):
        img_path = cls_imgs[cls_id]

        results = model(img_path)[0]
        plotted = results.plot()

        plotted = cv2.cvtColor(plotted, cv2.COLOR_BGR2RGB)

        plt.subplot(2, 2, idx)
        plt.imshow(plotted)
        plt.title(f"Case {cls_id+1}")
        plt.axis("off")

    plt.tight_layout()
    plt.show()



# 라벨 없는 이미지 찾기
def get_unlabeled_images(image_dir, label_dir, max_count=4):
    all_images = [f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".png"))]

    unlabeled = []

    for img_name in all_images:
        label_name = img_name.rsplit(".", 1)[0] + ".txt"
        label_path = os.path.join(label_dir, label_name)

        if not os.path.exists(label_path):
            img_path = os.path.join(image_dir, img_name)
            unlabeled.append(img_path)

    if len(unlabeled) > max_count:
        unlabeled = random.sample(unlabeled, max_count)

    return unlabeled

# 시각화 함수 (라벨 없는 이미지 출력)
def show_unlabeled_images(image_dir, label_dir, title="Unlabeled Images"):
    unlabeled_imgs = get_unlabeled_images(image_dir, label_dir, max_count=4)

    if len(unlabeled_imgs) == 0:
        print("❗ 라벨이 없는 이미지가 없습니다.")
        return

    plt.figure(figsize=(12, 10))
    plt.suptitle(title, fontsize=18)

    for idx, img_path in enumerate(unlabeled_imgs, 1):
        img = plt.imread(img_path)

        plt.subplot(2, 2, idx)
        plt.imshow(img)
        plt.title(f"Unlabeled {idx}")
        plt.axis("off")

    plt.tight_layout()
    plt.show()




# best.pt 성능 출력
def evaluate_split(split_name, size):
    if split_name == "valid":
        split_name = "val"

    print(f"\n==================== {split_name.upper()} EVALUATION ====================")

    data_yaml = os.path.join(dataset.location, "data.yaml")

    metrics = model.val(
        data=data_yaml,
        split=split_name,
        imgsz=size,
        save=False
    )

    precision = float(metrics.box.mp)
    recall    = float(metrics.box.mr)
    map50     = float(metrics.box.map50)
    map50_95  = float(metrics.box.map)

    print(f"{split_name} Precision: {precision:.4f}")
    print(f"{split_name} Recall:    {recall:.4f}")
    print(f"{split_name} mAP50:     {map50:.4f}")
    print(f"{split_name} mAP50-95:  {map50_95:.4f}")

In [ ]:
# YOLO 모델 로드

model = YOLO("best.pt")

In [ ]:
# Train / Valid / Test 평가

for size in [1024]:
    print(f"=== Evaluating with size {size} ===")
    evaluate_split("train", size)
    evaluate_split("val", size)
    evaluate_split("test", size)

In [ ]:
# Train / Valid / Test 시각화(라벨링된 이미지)

# show_by_class(train_img_dir, train_lbl_dir, "Train – 4 Different Classes")
# show_by_class(valid_img_dir, valid_lbl_dir, "Valid – 4 Different Classes")
show_by_class(test_img_dir,  test_lbl_dir,  "Test – 4 Different Classes")

In [ ]:
# Test 시각화(라벨링 안된 이미지)

show_unlabeled_images(test_img_dir, test_lbl_dir, "Test – Unlabeled Images")

In [ ]:
import os
import yaml
from collections import Counter
import matplotlib.pyplot as plt


# data.yaml에서 클래스 이름 로드
yaml_path = os.path.join(dataset.location, "data.yaml")
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

class_names = data["names"]
num_classes = len(class_names)

print("클래스 목록:", class_names)


# train/valid/test 전체 라벨에서 count 집계
label_dirs = [
    os.path.join(dataset.location, "train", "labels"),
    os.path.join(dataset.location, "valid", "labels"),
    os.path.join(dataset.location, "test", "labels")
]

cls_counter = Counter()

for label_dir in label_dirs:
    if not os.path.exists(label_dir):
        continue

    for lbl_file in os.listdir(label_dir):
        if not lbl_file.endswith(".txt"):
            continue

        with open(os.path.join(label_dir, lbl_file), "r") as f:
            lines = f.readlines()

        for line in lines:
            if line.strip() == "":
                continue

            cls_id = int(line.split()[0])
            cls_counter[cls_id] += 1



# classes, counts 자동 생성
classes = [class_names[cid] for cid in sorted(cls_counter.keys())]
counts  = [cls_counter[cid] for cid in sorted(cls_counter.keys())]

print("\n클래스별 객체 수:")
for cls, count in zip(classes, counts):
    print(f"{cls}: {count}")


# Bar Plot 시각화
colors = {
    "street_light": "#87CEFA",
    "illegal_parking": "green",
    "safety_fence": "red",
    "damage_road": "orange"
}

plt.figure(figsize=(8,5))
plt.bar(classes, counts, color=[colors[c] for c in classes])
plt.xlabel("Class Name", fontsize = 15, labelpad = 12)
plt.ylabel("Count", fontsize = 15)
plt.title("Object Counts by Class", fontsize = 20)
plt.xticks(rotation=0, fontsize = 12)
plt.grid(linewidth = 0.3)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# CSV 파일 불러오기
df = pd.read_csv("results.csv")

# 검증 성능 그래프
plt.figure(figsize=(8, 5))
plt.plot(df["epoch"], df["metrics/mAP50(B)"], label="mAP@50")
plt.plot(df["epoch"], df["metrics/mAP50-95(B)"], label="mAP@50-95")
plt.plot(df["epoch"], df["metrics/precision(B)"], label="Precision")
plt.plot(df["epoch"], df["metrics/recall(B)"], label="Recall")
plt.title("Validation Metrics per Epoch", fontsize = 20)
plt.xlabel("Epoch", fontsize = 15)
plt.ylabel("Score", fontsize = 15)
plt.xlim(0,)
plt.ylim(0,1)
plt.legend()
plt.grid(True)
plt.show()



plt.figure(figsize=(8, 5))

box_train   = "#1f77b4"
box_val     = "#4a90d9"

class_train = "#ff7f0e"
class_val   = "#ff9e47"

dfl_train   = "#2ca02c"
dfl_val     = "#66c266"

# Training
plt.plot(df["epoch"], df["train/box_loss"],  label="Training Box Loss",
         color=box_train, linestyle='-', linewidth=2)

plt.plot(df["epoch"], df["train/cls_loss"],  label="Training Class Loss",
         color=class_train, linestyle='-', linewidth=2)

plt.plot(df["epoch"], df["train/dfl_loss"],  label="Training DFL Loss",
         color=dfl_train, linestyle='-', linewidth=2)

# Validation
plt.plot(df["epoch"], df["val/box_loss"],    label="Validation Box Loss",
         color=box_val, linestyle='--', linewidth=2)

plt.plot(df["epoch"], df["val/cls_loss"],    label="Validation Class Loss",
         color=class_val, linestyle='--', linewidth=2)

plt.plot(df["epoch"], df["val/dfl_loss"],    label="Validation DFL Loss",
         color=dfl_val, linestyle='--', linewidth=2)

plt.xlabel("Epoch", fontsize=15)
plt.ylabel("Loss", fontsize=15)
plt.grid(True)
plt.legend()
plt.xlim(0,)
plt.ylim(0,)
plt.tight_layout()
plt.show()